In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pylab, mlab, pyplot
from scipy.misc import *
import h5py
from pylab import *
from numpy import *
from IPython.core.pylabtools import figsize, getfigs
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion
from scipy import signal as sg
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float

In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'C:\\Users\\fires\\Desktop\\CSIRO\\thursday\\data.h5', 'r')

images = np.asarray(data['images'])
labels = data['labels']

In [3]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]

In [4]:
# Reducing Image Size
images_cut = crop_center(images,66)
images_cut = np.asarray(images_cut)

In [5]:
count = 0

# convolving images in both x and y directions
for i, image in enumerate(images):
    # horizonal gradient of image
    grad_h = sg.convolve2d(image, [[-1, 0, 1],
                                      [-1, 0, 1],
                                      [-1, 0, 1]])
    # vertical gradient of image
    grad_y = sg.convolve2d(image, [[-1, -1, -1],
                                      [0, 0, 0],
                                      [1, 1, 1]])
    
    # combining x and y gradients by calculating the magnitude
    grad_mag = np.sqrt(np.power(grad_h, 2)+np.power(grad_y, 2))
    
    if count == 0:
        images_mag = np.expand_dims(grad_mag, axis=0)
        
    else:
        images_mag = np.vstack((images_mag,  np.expand_dims(grad_mag, axis=0)))
        
    count += 1  

In [6]:
mean_grads = []

# finding the average gradient magnitude for each image
for i, image in enumerate(images_mag):
    mean = np.mean(image)
    mean_grads.append(mean)
    
mean_grads = np.asarray(mean_grads)

In [7]:
FR1_means = []
FR2_means = []

# sorting means into FR1 and FR2 groups
for i, mean in enumerate(mean_grads):
    if labels[i] == False:
        FR1_means.append(mean)
    elif labels[i] == True:
        FR2_means.append(mean)
    else:
        print ("error")

# finding the mean mean for each class
FR1_mean = np.mean(np.asarray(FR1_means))
FR2_mean = np.mean(np.asarray(FR2_means))

# total mean
mean_total = (FR1_mean + FR2_mean)/2

In [14]:
tp = 0
tn = 0
fn = 0
fp = 0

# classifing by comparing image mean to total mean
for i, mean in enumerate(mean_grads):
    #if the image mean is above the mean_total then prediction is FR-II
    #if the image mean is below the mean_total then prediction is FR-I  
    
    #true = FR-II
    #false = FR-1
    if mean >  mean_total and labels[i] == True:
        tp += 1
         
    elif mean >  mean_total and labels[i] == False:
        fp += 1
    
    elif mean <  mean_total and labels[i] == True:
        fn += 1
    
    elif mean <  mean_total and labels[i] == False:
        tn += 1

# calculating balanced accuracy
accuracy_bal = (((tp)/len(FR2_means)) + ((tn)/len(FR1_means)))/2

print ("Balanced Accuracy: " + str(accuracy_bal))

recall = tp/(tp+fn)
print ("Recall: " + str(recall))

accuracy = (tp+tn)/np.shape(images)[0]
print ("Accuracy: " + str(accuracy))

F1 = (2*tp)/(2*tp+fp+fn)
print ("F1 Score: " + str(F1))

conf = np.zeros((2, 2))
conf[0, 0] = tp
conf[0, 1] = fp
conf[1, 0] = tn
conf[1, 1] = fn

print ("Confusion Matrix")
print (conf)
       

try:
    precision = tp/(tp+fp)
    print ("Precision: " + str(precision))

except:
    print ('Precision not defined')

Balanced Accuracy: 0.5329390418367703
Recall: 0.35772357723577236
Accuracy: 0.5870786516853933
F1 Score: 0.37446808510638296
Confusion Matrix
[[  44.   68.]
 [ 165.   79.]]
Precision: 0.39285714285714285


In [ ]:
"""
for n in range(0, images_mag.shape[0]):
   pyplot.imshow(images_mag[n,:,:], cmap='gray', shape=(267, 267))
   print (n)
   print (labels[n])
   plt.show()
"""